In [ ]:
using LinearAlgebra
using Plots
using FFTW

In [ ]:
include("tools.jl")

## Overview

Solution of Poisson's equation:

$$
\nabla^2 \phi(r) = -4\pi n(r).
$$

as the single line of code

`ϕ=cI(Linv(-4*π*O(cJ(n))));`

### Indexing

Computational implementation of problems in $d=3$ dimensions requires careful mapping of three-dimensional objects into a linear space. In lecture, we developed an approach to this indexing through the formation of two index matrices $M$ and $N$, where

$$M = \begin{bmatrix}
    0 & 0 & 0 \\
    1 & 0 & 0 \\
    \vdots & 0 & 0 \\
    S_1-1 & 0 & 0 \\
    S_1 & 0 & 0 \\
    \vdots & 1 & 0 \\
    \vdots & \vdots & 0 \\
    \vdots & S_2-1 & 0 \\
    \vdots & S_2 & 0 \\
    \vdots & \vdots & 1 \\
    \vdots & \vdots & \vdots \\
    \vdots & \vdots & S_3-1 \\
    \vdots & \vdots & S_3 \\
\end{bmatrix}
\qquad
N = \begin{bmatrix}
    0 & 0 & 0 \\
    1 & 0 & 0 \\
    \vdots & 0 & 0 \\
    -2 & 0 & 0 \\
    -1 & 0 & 0 \\
    \vdots & 1 & 0 \\
    \vdots & \vdots & 0 \\
    \vdots & -2 & 0 \\
    \vdots & -1 & 0 \\
    \vdots & \vdots & 1 \\
    \vdots & \vdots & \vdots \\
    \vdots & \vdots & -2 \\
    \vdots & \vdots & -1 \\
\end{bmatrix}$$

As a specific example, if `S=[3;3;2]`, then

$$M = \begin{bmatrix}
    0 & 0 & 0 \\
    1 & 0 & 0 \\
    2 & 0 & 0 \\
    0 & 1 & 0 \\
    1 & 1 & 0 \\
    2 & 1 & 0 \\
    0 & 2 & 0 \\
    1 & 2 & 0 \\
    2 & 2 & 0 \\
    0 & 0 & 1 \\
    1 & 0 & 1 \\
    2 & 0 & 1 \\
    0 & 1 & 1 \\
    1 & 1 & 1 \\
    2 & 1 & 1 \\
    0 & 2 & 1 \\
    1 & 2 & 1 \\
    2 & 2 & 1 \\
\end{bmatrix}$$

In [ ]:
function indexMN(S)
    ms = [i for i in 0:prod(S)-1]
    m1 = @. rem(ms, S[1])
    m2 = @. rem(div(ms, S[1]), S[2])
    m3 = @. rem(div(ms, S[1]*S[2]), S[3])
    M = [m1 m2 m3]
        
    n1 = @. m1 - (m1 > S[1]/2)*S[1]
    n2 = @. m2 - (m2 > S[2]/2)*S[2]
    n3 = @. m3 - (m3 > S[3]/2)*S[3]
    N = [n1 n2 n3]
    
    M, N
end

### Sampling points and reciprocal lattice vectors

In [ ]:
function GRSampling(R, S)
    M, N = indexMN(S)
    G = N * 2π*inv(R)
    r = M * inv(Diagonal(S)) * R' # R column vector [R1 R2 R3]
    
    G, r
end

In [ ]:
d2(r) = sum(r .^ 2, dims=2)[:]

In [ ]:
# size specification
S = [20;25;30]
R = Diagonal([6;6;6])
G, r = GRSampling(R, S)
G2 = d2(G);

In [ ]:
_,N = indexMN(S)
view(G2,S, N)


## Charge distribution

The charge density for our examle solution to Poisson's equation will be

$$n = g_1(r) - g_2(r) = 
\frac{e^{-r^2/(2\sigma_2^2)}}{(2\pi \sigma_2^2)^{3/2}} - 
\frac{e^{-r^2/(2\sigma_1^2)}}{(2\pi \sigma_1^2)^{3/2}}$$

where $r$ is the distance from the *center* of the cell, $g_1(r)$ and $g_2(r)$ are normalized three-dimensional Gaussian distributions, and $\sigma_1=0.75$ bohr and $\sigma_2=0.50$ bohr, respectively.
`

In [ ]:
char(r, σ) = exp(-r^2/(2*σ^2)) / (2π*σ^2)^(3/2)

In [ ]:
S = [20;25;30]
R = Diagonal([6;6;6])
G, r = GRSampling(R, S)
rmid = sum(R, dims=2)/2
rmids = ones(prod(S), 1) * rmid'
rd = r - rmids
rd = sqrt.(d2(rd))
n = char.(rd, 0.5) - char.(rd, 0.75);

In [ ]:
nx, ny, nz = 10, 12, 15
view(n, S, r, nx=nx, ny=ny, nz=nz)

## Operators

In [ ]:
struct DTs{fftT, ifftT}
    R::Matrix{Float64}
    G2::Vector{Float64}
    S::Vector{Int64}
    values::Array{Number}
    
    FFT::fftT
    IFFT::ifftT
end

function DTs(R, G2, S, values)  
    tmp = Array{Complex{Float64}}(undef, S...)
    FFT = plan_fft!(tmp)
    IFFT = plan_ifft!(tmp)
    
    DTs{typeof(FFT), typeof(IFFT)}(R, G2, S, values, FFT, IFFT)
end

In [ ]:
import Base: *, real, adjoint,
    broadcast

function *(x::Number, dts::DTs)
    res = x * dts.values
    DTs(dts.R, dts.G2, dts.S, res)
end

function *(dts::DTs, x::Number)
    res = x * dts.values
    DTs(dts.R, dts.G2, dts.S, res)
end

function *(dts::DTs, x::AbstractMatrix)
    res = x .* dts.values
    DTs(dts.R, dts.G2, dts.S, res)
end

function *(dts1::DTs, dts2::DTs)
    res = dts1.values * dts2.values
    DTs(dts1.R, dts1.G2, dts1.S, res)
end

function real(dts::DTs)
    res = real(dts.values)
    DTs(dts.R, dts.G2, dts.S, res)
end

function adjoint(dts::DTs)
    res = adjoint(dts.values)
    DTs(dts.R, dts.G2, dts.S, res)
end

In [ ]:
S = [20;25;30]
R = Matrix(Diagonal([6;6;6]))
G, r = GRSampling(R, S)
G2 = d2(G)[:];

In [ ]:
function O(in)
    out = det(in.R) .* in.values
    DTs(in.R, in.G2, in.S, out)
end

In [ ]:
function L(in)
    out = -det(in.R) * in.values .* in.G2
    DTs(in.R, in.G2, in.S, out)
end

**Check**: 

In [ ]:
in = DTs(R, G2, S, randn(prod(S)));
out = L(in)
[out.values ./ in.values -det(R)*G2][1:5, :]

In [ ]:
function Linv(in)
    out = - 1/det(in.R) * in.values ./ G2
    out[1] = 0.0
    DTs(in.R, in.G2, in.S, out)
end

**Check**:

In [ ]:
in = DTs(R, G2, S, randn(prod(S)));
out = Linv(L(in))
res = out.values ./ in.values
res[1:5]

In [ ]:
# fourier to real
function cI(in)
    out = reshape(in.FFT * reshape(in.values, in.S...), prod(in.S))
    DTs(in.R, in.G2, in.S, out)
end

# real to fourier
function cJ(in)
    out = reshape(in.IFFT * reshape(in.values, in.S...), prod(in.S))
    DTs(in.R, in.G2, in.S, out)
end

**Check**:

In [ ]:
in = DTs(R, G2, S, randn(prod(S)));
out = cJ(cI(in)) 
res = out.values ./ in.values
res[1:5]

In [ ]:
nin = DTs(R, G2, S, n);
ϕ=cI(Linv(-4*π*O(cJ(nin))));
ϕ = real(ϕ);

In [ ]:
nx, ny, nz = 10, 12, 15
view(ϕ.values, S, r, nx=nx, ny=ny, nz=nz)

In [ ]:
Unum = 0.5 * real(cJ(ϕ)' * O(cJ(nin)))
Unum.values

In [ ]:
σ1, σ2 = 0.75, 0.5
Uanal = ((1/σ1+1/σ2)/2 - √2 / √(σ1^2 + σ2^2)) / √π

### Ewald energy calculator

In [ ]:
S = [64;64;64]
R = Diagonal([16.0;16.0;16.0])
G, r = GRSampling(R, S)
G2 = d2(G);

In [ ]:
X = [0 0 0; 4.0 0 0];
Z = 1;
Sf = sum(cis.(G*X'), dims=2);

In [ ]:
rmid = sum(R, dims=2)/2.0
rmids = zeros(prod(S), 3)
for i in 1:3
    rmids[:,i] .= rmid[i]
end
rd = r - rmids;

In [ ]:
rd = sqrt.(d2(rd));

In [ ]:
gin_values = char.(rd, 0.25)
gin = DTs(R, G2, S, gin_values);
n = cI(cJ(gin)*Sf);
n = real(n);

In [ ]:
nx, ny, nz = 32,32,32
view(n.values, S, r, nx=nx, ny=ny, nz=nz)

In [ ]:
ϕ=cI(Linv(-4*π*O(cJ(n))));
ϕ = real(ϕ);
Unum = 0.5 * real(cJ(ϕ)' * O(cJ(n)))
Unum.values

How to calculate the ewald energy analysisly?